## Generating 'average waiting time in units service time' tables

This notebook uses the multi thread engine of the OpenQTSim package to start up multiple processes for the parallel processing of a list of tasks. 

This particular example notebook produces the typical 'average waiting time in units service time' tables that are presented in the PIANC 2014 guideline for example. You can experiment with different distributions for the interarrival and service times. Currently implemented are:
* 'M': Exponential distribution, 
* 'E2': Erlang 2 distribution.

NB: the table becomes more accurate when a larger number of arrivals (nr_arr) is simulated. Obviously there is a tradeoff between accuracy and calculation time.

### Create task list and perform calculations

In [8]:
%%time
if __name__ == '__main__': # protect your program's entry point
    
    import pandas as pd
    import numpy as np
    import openqtsim

    from multiprocessing import Pool
    from openqtsim.mt_engine import worker, Task

    # every instance will be run with nr_arr number of customers
    nr_arr = 20000     # nr of customers to simulate
    A = 'E2'            # distribution of inter arrival times (M: exponential, E2: erlang 2)
    S = 'E2'            # distribution of service times (M: exponential, E2: erlang 2)
    nr_servers = 10    # nr of servers to consider (1 to nr_servers)

    # --- servers to loop through ---
    # make an array of the nr of servers to loop through
    servers =  np.array(range(1, nr_servers + 1))

    # --- utilisations to loop through ---
    # lambdas are selected to be ranging from 1 to 9
    lambdas = np.array(range(1,10))

    # mu is selected to always be 10 
    mu=10
    
    # the lambdas divided by mu provide the utilisations to loop through
    utilisations = lambdas / mu

    tasks=[]
    # one by one add tasks to the 'tasks' list
    for index, lam in enumerate(lambdas):
        for c in servers:
            tasks.append(Task(A, S, c, nr_arr, lam, mu))

    # select nr of processes that will carry out tasks submitted to Pool
    p = Pool(8)
    
    # map the worker with tasks list to the Pool 
    result = p.map(worker, tasks)
    
    # close processes in the Pool
    p.close()

Wall time: 2min 52s


### Reshape 'result' variable to the desired table format

In [9]:
# organise results into a table of waiting times as a factor of service time
df = pd.DataFrame(tasks)
df['result'] = result
df['utilisation'] = df.lam / df.mu

df.pivot('utilisation', 'c', values='result')

c,1,2,3,4,5,6,7,8,9,10
utilisation,,,,,,,,,,
0.1,0.016607,0.000508,0.000020,0.000014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.2,0.061220,0.006404,0.000714,0.000268,0.000036,0.000000,0.000000,0.000000,0.000000,0.000000
0.3,0.127068,0.021829,0.006125,0.001543,0.000635,0.000264,0.000084,0.000002,0.000007,0.000000
0.4,0.225741,0.055926,0.018852,0.007154,0.003869,0.001484,0.000750,0.000355,0.000243,0.000028
0.5,0.381134,0.110335,0.053164,0.025108,0.012882,0.009549,0.004219,0.003340,0.001768,0.001795
0.6,0.599180,0.210165,0.117826,0.056855,0.042584,0.025391,0.018067,0.010744,0.008714,0.009060
0.7,1.073455,0.402193,0.228370,0.144858,0.110596,0.060349,0.050296,0.039639,0.032261,0.024097
0.8,1.934187,0.798264,0.461749,0.314386,0.278396,0.170074,0.139822,0.133593,0.101700,0.082349
0.9,5.658168,2.316400,1.286797,0.845577,0.800647,0.587989,0.626068,0.531610,0.381052,0.393690
